# Gate Assignment Problem (GAP) with PyHUBO

## Problem Overview

The Gate Assignment Problem (GAP) is a classic combinatorial optimization problem encountered in airport operations. In this example, we demonstrate how to model and solve GAP using PyHUBO's Higher-order Unconstrained Binary Optimization (HUBO) framework.

### Real-World Context

Airports need to assign arriving flights to available gates while minimizing passenger walking times and ensuring no two overlapping flights are assigned to the same gate. This is a constrained optimization problem where:

- **Variables**: Each flight needs to be assigned to exactly one gate
- **Objective**: Minimize total passenger walking time (passengers × walking distance to gate)
- **Constraints**: No two overlapping flights can use the same gate

### Problem Instance

In this example, we'll solve a simplified GAP with:
- **2 flights** that have overlapping time windows
- **4 available gates** with different walking distances from the terminal
- **Different passenger counts** for each flight

This tutorial will show you how to:
1. Define variable domains using `VariableDictionary`
2. Construct cost functions with `VariableAssignment`
3. Generate HUBO Hamiltonians
4. Solve using quantum annealing (OpenJIJ) and classical methods (PyQubo)

## Import PyHUBO Components

We'll import the three core PyHUBO classes needed for this example:

In [1]:

# Import PyHUBO core classes
from pyhubo import VariableAssignment  # Represents x_i = v_j assignments
from pyhubo import VariableDictionary    # Maps variables to binary representations
from pyhubo import HuboHamiltonian        # Generates HUBO coefficients

## Step 1: Define Variable Domains

The first step in PyHUBO is defining the **domain** of our combinatorial optimization problem. The domain specifies what values each variable can take.

### Problem Variables
- **Variables**: `flight_1`, `flight_2` (the flights to be assigned)
- **Values**: `gate_1`, `gate_2`, `gate_3`, `gate_4` (the available gates)

### VariableDictionary Class
The `VariableDictionary` automatically:
- Determines the minimum number of qubits needed for each variable
- Creates a unique binary encoding for each variable-value combination
- Provides mappings between logical values and binary representations

With 4 gates per flight, we need ⌈log₂(4)⌉ = 2 qubits per flight variable.

In [2]:
# Define the domain: each flight can be assigned to any of the 4 gates
domain = {"flight_1": ["gate_1", "gate_2", "gate_3", "gate_4"],
          "flight_2": ["gate_1", "gate_2", "gate_3", "gate_4"]}

# Create the variable dictionary
gap_variable_dict = VariableDictionary(domain)

# Let's examine the binary encoding
print("Variable encoding information:")
print(f"Total qubits needed: {gap_variable_dict.get_total_qubits()}")

# Every value for each variable has an index
print(f"Index for value 'gate_1' when assigned to flight 1: {gap_variable_dict.get_index('flight_1', 'gate_1')}")
print(f"I.e. when flight 1 is assigned to gate 1 then the variable flight_1 has the value {gap_variable_dict.get_index('flight_1', 'gate_1')}")

Variable encoding information:
Total qubits needed: 4
Index for value 'gate_1' when assigned to flight 1: 0
I.e. when flight 1 is assigned to gate 1 then the variable flight_1 has the value 0


## Step 2: Define the Cost Function

The cost function has two components:

### 1. Objective Function (Minimize Walking Time)
We want to minimize total passenger walking time: ∑(passengers × walking_time × assignment_indicator)

### 2. Constraint Penalty (Prevent Gate Conflicts) 
We add a penalty term to ensure overlapping flights don't use the same gate.

### VariableAssignment Class
`VariableAssignment(variable, value)` creates an indicator that equals 1 when `variable` is assigned `value`, and 0 otherwise. This is the core building block for constructing optimization expressions in PyHUBO.

**Mathematical formulation:**
- Objective: minimize ∑ᵢⱼ (passengersᵢ × walking_timeⱼ × xᵢⱼ)
- Constraint: ∑ⱼ xᵢⱼ × xₖⱼ = 0 for overlapping flights i,k and gate j

In [3]:
# Problem data: passenger counts and walking times
passengers = {
    "flight_1": 20,  # 20 passengers on flight 1
    "flight_2": 30   # 30 passengers on flight 2
}

walking_times_to_gate = {
    "gate_1": 50,  # minutes of walking time to gate 1
    "gate_2": 10,  # gate 2 is closest to terminal
    "gate_3": 15,  # moderate walking time
    "gate_4": 20   # moderate walking time
}

# Define which flights have overlapping time windows (can't use same gate)
overlapping_flights = [("flight_1", "flight_2")]

# Build the objective function: minimize total walking time
objective_function = 0

print("Building objective function terms:")
for flight_nbr, passenger_count in passengers.items():
    for gate_nbr, walking_time in walking_times_to_gate.items():
        cost = passenger_count * walking_time
        print(f"  {flight_nbr} → {gate_nbr}: cost = {passenger_count} × {walking_time} = {cost}")
        # Add term: cost × indicator_variable
        objective_function += VariableAssignment(flight_nbr, gate_nbr) * cost

print(f"\nObjective function: {objective_function}")

# Build constraint penalty: prevent overlapping flights from using same gate
penalty = 0
print("\nBuilding constraint penalty terms:")
for flight_pair in overlapping_flights:
    flight_i, flight_j = flight_pair
    print(f"  Preventing {flight_i} and {flight_j} from using the same gate:")
    for gate in walking_times_to_gate.keys():
        print(f"    Penalty for both using {gate}")
        # Add penalty: indicator_i × indicator_j (quadratic penalty)
        penalty += VariableAssignment(flight_i, gate) * VariableAssignment(flight_j, gate)

# Combine objective and penalty with Lagrange multiplier
lagrange_multiplier = 150  # Weight for constraint enforcement
cost_function = objective_function + lagrange_multiplier * penalty

print(f"\nTotal cost function: objective + {lagrange_multiplier} × penalty")
print(f"Cost function: {cost_function}")

Building objective function terms:
  flight_1 → gate_1: cost = 20 × 50 = 1000
  flight_1 → gate_2: cost = 20 × 10 = 200
  flight_1 → gate_3: cost = 20 × 15 = 300
  flight_1 → gate_4: cost = 20 × 20 = 400
  flight_2 → gate_1: cost = 30 × 50 = 1500
  flight_2 → gate_2: cost = 30 × 10 = 300
  flight_2 → gate_3: cost = 30 × 15 = 450
  flight_2 → gate_4: cost = 30 × 20 = 600

Objective function: 1000.0*Var(flight_1,gate_1) + 200.0*Var(flight_1,gate_2) + 300.0*Var(flight_1,gate_3) + 400.0*Var(flight_1,gate_4) + 1500.0*Var(flight_2,gate_1) + 300.0*Var(flight_2,gate_2) + 450.0*Var(flight_2,gate_3) + 600.0*Var(flight_2,gate_4)

Building constraint penalty terms:
  Preventing flight_1 and flight_2 from using the same gate:
    Penalty for both using gate_1
    Penalty for both using gate_2
    Penalty for both using gate_3
    Penalty for both using gate_4

Total cost function: objective + 150 × penalty
Cost function: 1000.0*Var(flight_1,gate_1) + 200.0*Var(flight_1,gate_2) + 300.0*Var(flight_1,

## Step 3: Generate the HUBO Hamiltonian

Now we combine the cost function with the variable dictionary to create a HUBO Hamiltonian. The `HuboHamiltonian` class automatically:

1. **Expands** `VariableAssignment` objects into binary variable products
2. **Collects** like terms to create polynomial coefficients  
3. **Maps** logical variables to physical qubits using the variable dictionary
4. **Generates** the final HUBO representation ready for quantum or classical solvers

The resulting Hamiltonian will be in the form:
$$H = \sum_i h_i Z_i + \sum_{i<j} J_{ij} Z_i Z_j + \sum_{i<j<k} K_{ijk} Z_i Z_j Z_k + \ldots$$

where $Z_i$ are Pauli-Z operators (or binary variables in classical context).

In [4]:
# Create the HUBO Hamiltonian
gap_hubo_hamiltonian = HuboHamiltonian(cost_function, gap_variable_dict)

print("HUBO Hamiltonian created successfully!")
print(f"Total qubits: {gap_variable_dict.get_total_qubits()}")
print("Ready to generate coefficients and solve the optimization problem.")

HUBO Hamiltonian created successfully!
Total qubits: 4
Ready to generate coefficients and solve the optimization problem.


## Step 4: Extract HUBO Coefficients

The `get_hamiltonian()` method returns a dictionary containing all the polynomial coefficients. Each key represents a product of qubits, and each value is the corresponding coefficient.

In [5]:
# Get the HUBO coefficients dictionary
hamiltonian_dict = gap_hubo_hamiltonian.get_hamiltonian()

print("HUBO Hamiltonian coefficients:")
print("=" * 40)
for terms, coefficient in hamiltonian_dict.items():
    terms = [i for i in terms]
    if len(terms) == 0:
        print(f"Constant term: {coefficient}")
    elif len(terms) == 1:
        print(f"Linear Z_{terms[0]}: {coefficient}")
    elif len(terms) == 2:
        print(f"Quadratic Z_{terms[0]} × Z_{terms[1]}: {coefficient}")
    else:
        term_str = " × ".join([f"Z_{t}" for t in terms])
        print(f"Higher-order {term_str}: {coefficient}")

print(f"\nTotal number of terms: {len(hamiltonian_dict)}")

HUBO Hamiltonian coefficients:
Constant term: 1225.0
Linear Z_('flight_1', 0): 175.0
Linear Z_('flight_1', 1): 125.0
Quadratic Z_('flight_1', 0) × Z_('flight_1', 1): 225.0
Linear Z_('flight_2', 0): 262.5
Linear Z_('flight_2', 1): 187.5
Quadratic Z_('flight_2', 0) × Z_('flight_2', 1): 337.5
Quadratic Z_('flight_2', 0) × Z_('flight_1', 0): 37.5
Quadratic Z_('flight_2', 1) × Z_('flight_1', 1): 37.5
Higher-order Z_('flight_2', 0) × Z_('flight_2', 1) × Z_('flight_1', 1) × Z_('flight_1', 0): 37.5

Total number of terms: 10


## Understanding the Hamiltonian Structure

The generated Hamiltonian has the form:
$$H = 1112.5 \mathbb{I} + 175 \hat{Z}_{1,0} + 125 \hat{Z}_{1,1} + 225 \hat{Z}_{1,0} \hat{Z}_{1,1} + \cdots$$

Where:
- $\mathbb{I}$ is the identity (constant offset)
- $\hat{Z}_{i,j}$ is a Pauli-Z operator on qubit $j$ of variable $i$
- $(i,j) = (1,0)$ refers to the least significant bit of flight_1's gate assignment
- $(i,j) = (1,1)$ refers to the most significant bit of flight_1's gate assignment

**Qubit Index Convention:**
- Qubits 0,1: encode flight_1's gate assignment
- Qubits 2,3: encode flight_2's gate assignment
- Binary encoding: 00→gate_1, 01→gate_2, 10→gate_3, 11→gate_4

## Step 5: Validate Solutions

Before solving with quantum/classical annealers, let's verify that our Hamiltonian correctly evaluates known solutions. We'll test a proposed solution where flight_1 → gate_3 and flight_2 → gate_2.

In [6]:
# Test a proposed solution
proposed_solution = {"flight_1": "gate_3",
                    "flight_2": "gate_2"}

# Calculate the cost using the HUBO Hamiltonian
cost = gap_hubo_hamiltonian.cost_solution(proposed_solution)
print(f"Cost of proposed solution: {cost}")

# Verify by manual calculation
manual_cost = (passengers["flight_1"] * walking_times_to_gate["gate_3"] + 
               passengers["flight_2"] * walking_times_to_gate["gate_2"])
print(f"Manual calculation: {passengers['flight_1']} × {walking_times_to_gate['gate_3']} + "
      f"{passengers['flight_2']} × {walking_times_to_gate['gate_2']} = {manual_cost}")

# Check if flights conflict (both at same gate)
conflict = proposed_solution["flight_1"] == proposed_solution["flight_2"]
print(f"Gate conflict: {conflict}")

Cost of proposed solution: 600.0
Manual calculation: 20 × 15 + 30 × 10 = 600
Gate conflict: False


## Step 6: Solving with OpenJIJ and PyQubo

This section demonstrates how PyHUBO integrates with popular optimization libraries:

1. **OpenJIJ**: Direct HUBO solving using simulated annealing
2. **PyQubo + Neal**: Convert to QUBO format and solve with D-Wave's classical solver

### Why Two Approaches?

- **HUBO (Higher-order)**: Natural problem representation, fewer auxiliary variables
- **QUBO (Quadratic)**: Compatible with D-Wave quantum annealers and many classical solvers

### Expected Optimal Solution

For our problem instance:
- Flight 1 (20 passengers) → Gate 3 (15 min walk) = 300 total walking time
- Flight 2 (30 passengers) → Gate 2 (10 min walk) = 300 total walking time
- **Total cost: 600** (no gate conflicts)

### Method 1: Direct HUBO Solving with OpenJIJ

In [7]:
# Helper function to convert qubit indices to string format for OpenJIJ
def map_coeff_to_str(coeffs):
    """Convert coefficient tuples to string format required by OpenJIJ"""
    return tuple([coeff[0] + "_" + str(coeff[1]) for coeff in coeffs])

# Export the Hamiltonian in OpenJIJ format
print("Converting HUBO Hamiltonian for OpenJIJ...")
hubo_hamiltonian_dict = gap_hubo_hamiltonian.export_dict(
    map_coeff_to_str, 
    non_standard_definition_pauli_z=True  # Use {-1, +1} spin variables
)

print(f"Exported {len(hubo_hamiltonian_dict)} HUBO terms")

# Solve using OpenJIJ's Simulated Annealing
import openjij as oj

print("Solving with OpenJIJ Simulated Annealing...")
sampler = oj.SASampler()
response = sampler.sample_hubo(
    hubo_hamiltonian_dict, 
    vartype="SPIN",      # Use spin variables {-1, +1}
    num_reads=100,       # Number of optimization runs
    num_sweeps=1000      # Annealing steps per run
).change_vartype("BINARY")  # Convert to binary {0, 1} for interpretation

print("OpenJIJ Results:")
print("=" * 50)
print(response)

# Extract the best solution
best_sample = response.first.sample
best_energy = response.first.energy
print(f"\nBest energy found: {best_energy}")
print(f"Best binary assignment: {dict(best_sample)}")

Converting HUBO Hamiltonian for OpenJIJ...
Exported 10 HUBO terms
Solving with OpenJIJ Simulated Annealing...
OpenJIJ Results:
   flight_1_0 flight_1_1 flight_2_0 flight_2_1 energy num_oc.
0           0          1          1          0  600.0       1
1           0          1          1          0  600.0       1
2           0          1          1          0  600.0       1
4           0          1          1          0  600.0       1
6           0          1          1          0  600.0       1
7           0          1          1          0  600.0       1
8           0          1          1          0  600.0       1
9           0          1          1          0  600.0       1
10          0          1          1          0  600.0       1
12          0          1          1          0  600.0       1
14          0          1          1          0  600.0       1
15          0          1          1          0  600.0       1
16          0          1          1          0  600.0       1
17   

### Interpreting OpenJIJ Results

OpenJIJ found binary solutions where:
- `(01)₂` for flight_1 corresponds to gate assignment
- `(10)₂` for flight_2 corresponds to gate assignment

Let's convert these binary strings back to meaningful gate assignments using the `VariableDictionary`:

In [8]:
# Convert OpenJIJ binary solution to string format
# Note: This will need to be updated based on actual OpenJIJ output
openjij_solution = {"flight_1": "01",  # Binary string for flight_1
                    "flight_2": "10"}  # Binary string for flight_2

# Convert binary strings back to gate assignments
interpreted_solution = gap_variable_dict.string_format_to_solution_dict(openjij_solution)
print("OpenJIJ Solution Interpretation:")
print("=" * 35)
for flight, gate in interpreted_solution.items():
    print(f"{flight} → {gate}")

# Verify this matches our expected optimal solution
print(f"\nSolution cost verification:")
cost = gap_hubo_hamiltonian.cost_solution(interpreted_solution)
print(f"Total cost: {cost}")

# Manual verification
flight1_cost = passengers["flight_1"] * walking_times_to_gate[interpreted_solution["flight_1"]]
flight2_cost = passengers["flight_2"] * walking_times_to_gate[interpreted_solution["flight_2"]]
print(f"Flight 1 cost: {passengers['flight_1']} × {walking_times_to_gate[interpreted_solution['flight_1']]} = {flight1_cost}")
print(f"Flight 2 cost: {passengers['flight_2']} × {walking_times_to_gate[interpreted_solution['flight_2']]} = {flight2_cost}")
print(f"Total: {flight1_cost + flight2_cost}")

OpenJIJ Solution Interpretation:
flight_1 → gate_3
flight_2 → gate_2

Solution cost verification:
Total cost: 600.0
Flight 1 cost: 20 × 15 = 300
Flight 2 cost: 30 × 10 = 300
Total: 600


✅ **OpenJIJ successfully found the optimal solution!**

The solver correctly identified that assigning:
- Flight 1 → Gate 3 (cost: 20 × 15 = 300)  
- Flight 2 → Gate 2 (cost: 30 × 10 = 300)

Gives the minimum total walking time of 600 minutes with no gate conflicts.

### Method 2: QUBO Conversion with PyQubo + Neal

Now let's demonstrate how PyHUBO can convert problems to QUBO format for compatibility with D-Wave systems and other QUBO solvers. This approach requires adding one-hot constraints to ensure valid assignments.

In [9]:
# Convert PyHUBO cost function to PyQubo format
print("Converting to PyQubo format...")
pyqubo_cost_function = cost_function.to_pyqubo()
print(f"PyQubo cost function: {type(pyqubo_cost_function)}")

# IMPORTANT: QUBO requires one-hot constraints
# Each variable must be assigned to exactly one value
print("\nAdding one-hot constraints...")
pyqubo_one_hot_penalty = gap_variable_dict.one_hot_penalty()
print(f"One-hot penalty terms: {pyqubo_one_hot_penalty}")

# Combine objective with one-hot constraints
one_hot_lagrange_penalty = 400  # Weight for one-hot enforcement
print(f"One-hot penalty weight: {one_hot_lagrange_penalty}")

total_model = pyqubo_cost_function + one_hot_lagrange_penalty * pyqubo_one_hot_penalty
print("Compiling PyQubo model...")
model = total_model.compile()
print("✅ PyQubo model compiled successfully!")

print(f"\nModel statistics:")
print(f"  Number of variables: {len(model.variables)}")
print(f"  Variables: {sorted(model.variables)}")

Converting to PyQubo format...
PyQubo cost function: <class 'cpp_pyqubo.Add'>

Adding one-hot constraints...
One-hot penalty terms: ((1.000000 * ((1.000000 + (-1.000000 * (Binary('flight_2_gate_4') + Binary('flight_2_gate_3') + Binary('flight_2_gate_2') + 0.000000 + Binary('flight_2_gate_1')))) * (1.000000 + (-1.000000 * (Binary('flight_2_gate_4') + Binary('flight_2_gate_3') + Binary('flight_2_gate_2') + 0.000000 + Binary('flight_2_gate_1')))))) + 0.000000 + (1.000000 * ((1.000000 + (-1.000000 * (Binary('flight_1_gate_4') + Binary('flight_1_gate_3') + Binary('flight_1_gate_2') + 0.000000 + Binary('flight_1_gate_1')))) * (1.000000 + (-1.000000 * (Binary('flight_1_gate_4') + Binary('flight_1_gate_3') + Binary('flight_1_gate_2') + 0.000000 + Binary('flight_1_gate_1')))))))
One-hot penalty weight: 400
Compiling PyQubo model...
✅ PyQubo model compiled successfully!

Model statistics:
  Number of variables: 8
  Variables: ['flight_1_gate_1', 'flight_1_gate_2', 'flight_1_gate_3', 'flight_1_ga

### Solving the QUBO Model

We'll use D-Wave's Neal simulated annealing sampler to solve the QUBO formulation:

In [10]:
# Solve using D-Wave Neal simulated annealing
import neal

print("Solving QUBO with Neal Simulated Annealing...")
sampler = neal.SimulatedAnnealingSampler()

# Convert to Binary Quadratic Model (BQM) format
bqm = model.to_bqm()
print(f"BQM created with {len(bqm.variables)} variables")

# Run the sampler
sampleset = sampler.sample(bqm, num_reads=100, num_sweeps=1000)
print(f"Sampling completed. {len(sampleset)} samples generated.")

# Decode the results back to logical variables
decoded_samples = model.decode_sampleset(sampleset)
print(f"Decoded {len(decoded_samples)} samples")

# Find the best (lowest energy) sample
best_sample = min(decoded_samples, key=lambda x: x.energy)

print("\nNeal/PyQubo Results:")
print("=" * 30)
print(f"Best energy: {best_sample.energy}")
print(f"Best sample: {best_sample.sample}")

# Check if the solution satisfies one-hot constraints
print(f"\nConstraint validation:")
for constraint, value in best_sample.constraints().items():
    if hasattr(value, 'keys') and 'penalty' in value:
        print(f"  {constraint}: penalty = {value['penalty']}")
    else:
        print(f"  {constraint}: {value}")

# Display the final gate assignments
print(f"\nFinal gate assignments:")
solution_dict = {}
for var, val in best_sample.sample.items():
    if val == 1:  # Only show activated variables
        # Parse variable name to extract flight and gate
        if 'flight' in var and 'gate' in var:
            parts = var.split('_')
            flight = f"{parts[0]}_{parts[1]}"
            gate = f"{parts[2]}_{parts[3]}"
            solution_dict[flight] = gate
            print(f"  {flight} → {gate}")

# Verify the solution cost
if solution_dict:
    verification_cost = gap_hubo_hamiltonian.cost_solution(solution_dict)
    print(f"\nSolution cost verification: {verification_cost}")

Solving QUBO with Neal Simulated Annealing...
BQM created with 8 variables
Sampling completed. 100 samples generated.
Decoded 100 samples

Neal/PyQubo Results:
Best energy: 600.0
Best sample: {'flight_1_gate_1': 0, 'flight_1_gate_3': 1, 'flight_1_gate_4': 0, 'flight_2_gate_2': 1, 'flight_2_gate_1': 0, 'flight_1_gate_2': 0, 'flight_2_gate_3': 0, 'flight_2_gate_4': 0}

Constraint validation:

Final gate assignments:
  flight_1 → gate_3
  flight_2 → gate_2

Solution cost verification: 600.0


## Summary and Conclusions

🎯 **Both solving methods successfully found the optimal solution:**

- **OpenJIJ (Direct HUBO)**: Native higher-order optimization without auxiliary variables
- **Neal + PyQubo (QUBO)**: Standard quadratic formulation compatible with D-Wave systems

### Key PyHUBO Features Demonstrated

1. **Natural Problem Modeling**: Define optimization problems using intuitive mathematical expressions
2. **Automatic Binary Encoding**: `VariableDictionary` handles qubit allocation and variable mapping
3. **Flexible Cost Functions**: Combine objectives and constraints using `VariableAssignment` objects
4. **Multiple Solver Integration**: Export to OpenJIJ (HUBO) and PyQubo (QUBO) formats
5. **Solution Validation**: Built-in cost evaluation and constraint checking